# Classify Stars in Colliding Galaxies

Using ~5000 labeled stars out of ~80,000 in each galaxy (170,000 stars total), make a model to predict and/or classify all the stars in the collection of the galaxy collision.

![KNN Classifier results for colliding galaxies](Assets/KNNPredictionResult.png "Classifying Stars in Colliding Galaxies")

Follow the creation of two synthetically created star locations in Euclidean coordinates. The galaxies are set to collide with each other. Techniques described are an approach to classifying which stars belong to which galaxy after the collision. We also later identify the coordinates of pairs of points that lie within a radius R of each other. Points already lying with radius R in either galaxy are ignored, and the focus is narrowed to points in one galaxy in close proximity to the points in the other galaxy.

We synthesize ~ 80,000 stars in each of two galaxies—a fictitious GFFA ("galaxy far, far away") and a fictitious "Xeno", which is the backdrop for Superman's Krypton. Combined, this is over 100,000 stars, for which we search for neighboring pairs on the order of a few light years apart.

Play the following video which plots and rotates the combined galaxies. It plots a 3D scatter plot of the nearly 200,000 stars in the two galaxies. It gradually reduces the opacity of each star to ultimately reveal the one or two or handfuls of stars that are within a given radius of each other as a result of the collision of the two galaxies. The center bulge in each galaxy is so dense that the red zone stars are not visible until we turn the opacity of the stars down significantly.


<video controls src="./Videos/CollidingGalaxiesAnimation.mp4" />



# Learning Objectives:
 
1. Apply scikit-learn kNN  (using k-d tree) and RandomForest to classify stars belonging to each galaxy within a combined super galaxy to determine most accurate model.
1. Apply Intel® Extension for Scikit-learn* patch and SYCL context to compute on available GPU resource.
1. **Optional**: Use KDtree, also known as, k-d tree to find all stars interacting within a given radius (say two light years) due to the collision of two galaxies.


To synthesized the galaxy data uses a parametric equations described in the following paper [A New Formula Describing the Scaffold Structure of Spiral Galaxies](https://arxiv.org/ftp/arxiv/papers/0908/0908.0892.pdf) regarding the parametric equation of arm: :


$$ r \left( \phi \right) = \frac{A}{log(B \  \ tan(\frac{\phi}{2N}))  } $$

The synthesizer used here, generates an arbitrary number of arms, generates a 3D gauassian distribution of stars around a galactic center, then distributes a gausian distribiutoin  of stars along the length of each arm

We also used rotation matrices  from this blog [3D Rotations and Euler angles in Python](https://www.meccanismocomplesso.org/en/3d-rotations-and-euler-angles-in-python/)


### Fictitious Galaxies: 

We create two fictitious galaxies: GFFA ("galaxy far, far away") and Xeno ("The purported galaxy for Superman's planet Krypton). We intersect the galaxies and use various classification algorithms to identify the stars in each galaxy.



In [ ]:
import math as m
import numpy as np
import random
import math as m
import matplotlib.pyplot as plt
def Rx(theta):
    return np.matrix([[ 1, 0           , 0           ],
                   [ 0, m.cos(theta),-m.sin(theta)],
                   [ 0, m.sin(theta), m.cos(theta)]])
  
def Ry(theta):
    return np.matrix([[ m.cos(theta), 0, m.sin(theta)],
                   [ 0           , 1, 0           ],
                   [-m.sin(theta), 0, m.cos(theta)]])
  
def Rz(theta):
    return np.matrix([[ m.cos(theta), -m.sin(theta), 0 ],
                   [ m.sin(theta), m.cos(theta) , 0 ],
                   [ 0           , 0            , 1 ]])

def ParametricPolar2Rectangular(r2, phi):
    x = r2*np.cos(phi) #+.08 * random.normal(loc=0, scale=.68, size=(NBlue))
    y = r2*np.sin(phi) #+.04 * random.normal(loc=0, scale=.34, size=(NBlue))
    z = np.zeros(len(x))
    return np.array([x,y,z]).T

def R(phi, A, B, N, Phase=0.0):
    return np.nan_to_num( A/np.log( B*np.tan(phi/(2.0*N))))

def createGalaxyArms(phi, arms = 6, A = 1.0, B = .69, N = 5, Radius = 4.0e5, 
                     bandMaxWidth = 80, GlobRadius = 3.0e4,GlobNumber = 2000, GlobZSquish = .1):
    rot = 360/arms
    band = []
    cumsum = list(np.cumsum(np.histogram(phi, bins = 10)[0]))
    percent = (np.histogram(phi)[0] / sum(np.histogram(phi, bins = 10)[0]))
    #print('cumsum ', cumsum)
    #print('percent ', percent)   
    L = len(percent)
    # each band indicates the width and number of stars to put into an arm at each band lavel
    # populate stars in a generic non locality specific way into each band
    for j in range(L):
        band.append(bandMaxWidth * (L-j)/100. * np.random.normal(loc=0, scale=1, size=( L-j,3)))
    for i in range (arms):
        # coompute location r, phi given this equation which is governed by paramater phi
        # A, B, N control the shape and length of an arm
        r2 = R(phi, A, B, N )
        # StatCenterCartesian eapnads our little galaxy to a rough radius say in light years
        StatCenterCartesian = Radius*ParametricPolar2Rectangular(r2, phi) 
        j = 0
        # loop thru all known star center/ centroids in a single arm, create a grouping of stars at each centroid 
        for k in range(len(phi)):
            # k ranges over the length of an arm
            # in each range of values of k, a different size and population of stars will be generated
            if k > cumsum[j]:
                j += 1
            Stars_onej = band[j].copy()
            # set z coordinate to zero for now
            Stars_onej[:,2] = 0
            # add the StatCenterCartesian to each star in the band by stacking the array vertically
            Stars_onej += StatCenterCartesian[k] # this needs work, this needs to be doen once per band not once per k
            if k == 0:
                Stars_oneArm = Stars_onej
            else:
                Stars_oneArm = np.vstack((Stars_oneArm,Stars_onej))
            # at this point a single galaxy arm has been populated
            # to get the other arms positions, we do a matrix rotation using Rz
            
        # add a few globular clusters randomly in each arm
        N_GlobularClusters = 4
        N_GlobularClusterStars = 100
        R_GlobularCluster = 300
        GlobularCLustersAtZero = R_GlobularCluster * np.random.normal(loc=0, scale=1, size=(N_GlobularClusterStars,3))
        GlobCOffset = np.random.uniform(low = -5*R_GlobularCluster,  high= 5*R_GlobularCluster, size=(N_GlobularClusters,3))

        phiGlobC = np.random.uniform(low= 0,  high= np.pi/2.0, size=(N_GlobularClusters,))
        r3 = R(phiGlobC, A, B, N )
        #print(ParametricPolar2Rectangular(r3, phiGlobC).shape, GlobCOffset.shape, GlobularCLustersAtZero.shape )
        RGlobC = Radius * ParametricPolar2Rectangular(r3, phiGlobC) + GlobCOffset
        GlobularCLustersAtR = []
        for gc in range(N_GlobularClusters):
            Stars_oneArm = np.vstack((np.array(Stars_oneArm), GlobularCLustersAtZero + RGlobC[gc]))
        Stars_eachArm = np.vstack((np.array(Stars_oneArm*Rz(i*rot*np.pi/180.)), RGlobC))

        
        #print(Stars_eachArm.shape)
        # keep rotations for StatCenterCartesian_eachArm as well
        StatCenterCartesian_eachArm = np.array(StatCenterCartesian*Rz(i*rot*np.pi/180.))
        # next we vertically stack all data for all arms into Stars_allArms
        if i == 0:
            Z_allArms = StatCenterCartesian_eachArm
            Stars_allArms = Stars_eachArm
        else:
            Z_allArms = np.vstack((Z_allArms,StatCenterCartesian_eachArm)) 
            Stars_allArms = np.vstack((Stars_allArms,Stars_eachArm)) 
        # next create a glob of stars at center fo galaxy
        CenterGlob = GlobRadius * np.random.normal(loc=0, scale=[1, 1, GlobZSquish], size=(GlobNumber,3))
        # Put ALL galaxy stars into Stars
        Stars = np.vstack((Stars_allArms , CenterGlob))
    return Stars_allArms, CenterGlob, Stars

def dist(p1, p2):
    diff = p1 - p2
    return np.sqrt(diff.dot(diff))

def distInd(tooClose, index1, index2):
    diff = tooClose[index1] - tooClose[index2]
    return np.sqrt(diff.dot(diff))

def unpack_sklearn_query_radii(Stars, neighbors, counts):
    tooClose = []
    for pair in neighbors[counts > 1]:
        tooClose.append(Stars[pair[0]])
        tooClose.append(Stars[pair[1]])
    return np.array(tooClose)

def write_dictionary(dictionary, fn):
    import pickle
    import os
    here = './'
    # Store data (serialize)
    with open(os.path.join(here,fn), 'wb') as handle:
        #pickle.dump(dictionary, handle, protocol=pickle.HIGHEST_PROTOCOL)
        pickle.dump(dictionary, handle)
    return

def read_dictionary(fn):
    import pickle
    # Load data (deserialize)
    with open(fn, 'rb') as handle:
        dictionary = pickle.load(handle)
    return dictionary

def add(arr, myset):
    for pt3D in arr:
        mylist = []
        for pt in pt3D:
            mylist.append(round(pt,0))
        if(tuple(mylist) not in myset):
            myset.add(tuple(mylist))
            
#write_dictionary(collision, 'collision.pkl')
#collision = read_dictionary('collision.pkl')
GalaxyFromFile = False

# Draw randomly generated stars for GFFA galaxy

In [ ]:
%matplotlib inline
if GalaxyFromFile == False:
    phi = np.abs(np.random.normal(loc=0, scale=.8, size=(600,1)))
    phi =  phi[phi<np.pi/2.]
    Arms, CenterGlob, Stars = createGalaxyArms(phi, arms = 5, Radius = 4.0e4, bandMaxWidth = 1.0e4, GlobRadius = 4e3, GlobNumber = 60000)

    GFFA = dict()
    GFFA['Arms'] = Arms.copy()
    GFFA['CenterGlob'] = CenterGlob.copy()
    GFFA['Stars'] = Stars.copy()
    print("GFFA['Stars'][:,0]", GFFA['Stars'].shape)

    GFFA['CenterGlob'] = np.vstack((GFFA['CenterGlob'],np.array([101,100,0])))
    GFFA['Stars'] = np.vstack((GFFA['Stars'],np.array([101,100,0])))

    plt.style.use('dark_background')
    plt.scatter(GFFA['Stars'][:,0], GFFA['Stars'][:,1], c = 'cyan', s = .1, alpha = .2)


# Notices & Disclaimers

Intel technologies may require enabled hardware, software or service activation.

No product or component can be absolutely secure.
Your costs and results may vary.

© Intel Corporation. Intel, the Intel logo, and other Intel marks are trademarks of Intel Corporation or its subsidiaries. 
*Other names and brands may be claimed as the property of others.
